In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
dbutils.widgets.text("target_dataset", "VCM_DWH_PRD") 
dbutils.widgets.text("target_table", "f_po_doctype")
dbutils.widgets.text("metadata_schema", "udp_wcm_metadata_dev")
dbutils.widgets.text(
    "dependency_table","VCM_STG_INF.SAP_VCDMMPO03" 
)

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
dbutils.widgets.text("field_calday", "CALDAY")
dbutils.widgets.text("field_id", "HASH_ID")

In [0]:
field_calday = dbutils.widgets.get("field_calday")
field_id = dbutils.widgets.get("field_id")

print(f"field_calday: {field_calday}")
print(f"field_id: {field_id}")

In [0]:
environment = dbutils.widgets.get("environment")
target_dataset = dbutils.widgets.get("target_dataset")
target_table = dbutils.widgets.get("target_table")
metadata_schema = dbutils.widgets.get("metadata_schema")
dependency_table = dbutils.widgets.get("dependency_table")
dependency_table = [x.strip().upper() for x in dependency_table.split(",")]
dependency_table = "'" + "','".join(dependency_table) + "'"

print(f"environment: {environment}")
print(f"target_dataset: {target_dataset}")
print(f"target_table: {target_table}")
print(f"catalog_name: {catalog_name}")
print(f"metadata_schema: {metadata_schema}")
print(f"dependency_table: {dependency_table}")

In [0]:
%run "../common/common_etl_load"

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.f_po_doctype
( 
  po_id STRING,
  po_doc STRING,
  po_item BIGINT,
  product_id STRING,
  store_id STRING,
  dc_id STRING,
  bu_id STRING,
  vendor STRING,
  doctype STRING,
  po_status STRING,
  calday DATE,
  po_unit STRING,
  creation_date DATE,
  pur_reason STRING,
  purch_org STRING,
  pur_group STRING,
  gr_date DATE,
  gr_re_ind STRING,
  compl_del STRING,
  base_unit STRING,
  promotion_code STRING,
  delivery_method STRING,
  po_value FLOAT,
  gr_value FLOAT,
  po_qty FLOAT,
  gr_qty FLOAT
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
if CALDAY_IN_STR != "":
    spark.sql(
        f"""
              CREATE OR REPLACE TEMP VIEW temp_{target_table} AS
              WITH MAIN AS (
                  SELECT 
                  po.oi_ebeln ||'_'|| po.oi_ebelp po_id,
                  po.oi_ebeln po_doc,
                  po.oi_ebelp po_item,
                  po.material product_id, 
                  po.plant store_id,
                  po.supp_plant dc_id,
                  po.segment bu_id, 
                  po.vendor,
                  po.doctype,
                  po.po_status, 
                  po.calday,
                  po.po_unit,
                  po.bic_zk_credat creation_date, 
                  po.pstng_date gr_date,
                  po.gr_re_ind gr_re_ind,
                  po.compl_del,
                  po.base_uom base_unit,
                  po.rt_promo promotion_code,
                  po.pur_reason,
                  po.purch_org,
                  po.pur_group,
                  po.proc_date,
                  CASE 
                      WHEN po.segment = '1100' THEN 'DC'
                      WHEN po.segment IN ('1500','2000') THEN 'DSD'
                      ELSE po.segment
                  END delivery_method,
                  SUM(po.sr_povlc) OVER (PARTITION BY po.oi_ebeln, po.oi_ebelp, po.bic_zk_credat) po_value,
                  SUM(po.sr_grvlc) OVER (PARTITION BY po.oi_ebeln, po.oi_ebelp, po.bic_zk_credat) gr_value,
                  SUM(po.sr_poqbu) OVER (PARTITION BY po.oi_ebeln, po.oi_ebelp, po.bic_zk_credat) po_qty, 
                  SUM(po.sr_grqbu) OVER (PARTITION BY po.oi_ebeln, po.oi_ebelp, po.bic_zk_credat) gr_qty,
                  ROW_NUMBER() OVER (
                    PARTITION BY po.oi_ebeln, po.oi_ebelp 
                    ORDER BY po.upd_date DESC, bic_zk_cretim DESC, sr_poqbu DESC
                    ) AS rn
                    FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vcdmmpo03 AS po
                    WHERE CALDAY IN ({CALDAY_IN_STR})
                    and (item_del != 'X' OR item_del IS NULL) 
                    AND (
                        segment IN ('1100', '1500', '2000', '1009')
                        OR plant IN (SELECT store_id FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store WHERE business_unit = 'FCDC')
                        OR supp_plant IN (SELECT store_id FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store WHERE business_unit = 'FCDC')
                        ) 
                        AND doctype IN ('NB','ZNBC','UB','ZUB')
                        )
                        SELECT
                            po_id,
                            po_doc,
                            po_item,
                            product_id,
                            store_id,
                            dc_id,
                            bu_id,
                            vendor,
                            doctype,
                            po_status,
                            calday,
                            po_unit,
                            creation_date,
                            pur_reason,
                            purch_org,
                            pur_group,
                            proc_date,
                            gr_date,
                            gr_re_ind,
                            compl_del,
                            base_unit,
                            promotion_code,
                            delivery_method,
                            po_value,
                            gr_value,
                            po_qty,
                            gr_qty	
                            FROM MAIN
                            WHERE rn = 1
                            """
                                )
    
else:
    print(f"No valid calday's found In common-etl to create the view: temp_{target_table}")

In [0]:
if CALDAY_IN_STR != '':
    calday_query = f"""
        SELECT 
            CONCAT("'", CONCAT_WS("','", COLLECT_SET(CAST(TO_DATE(calday, 'yyyy-MM-dd') AS STRING))), "'") AS calday_in_str
        FROM temp_{target_table}
        """
    calday_df = spark.sql(calday_query)
    calday_in_str = calday_df.collect()[0][0]
else:
    print(f"No valid calday's found In common-etl to create the view: temp_{target_table}")

In [0]:
if CALDAY_IN_STR != "":
    # Get the calday list string from temp table
    calday_query = f"""
    SELECT
        CONCAT("'", CONCAT_WS("','", COLLECT_SET(CAST(TO_DATE(calday, 'yyyy-MM-dd') AS STRING))), "'") AS calday_in_str
    FROM temp_{target_table}
    """

    calday_df = spark.sql(calday_query)
    calday_in_str = calday_df.collect()[0][0]

    # Only run DELETE if the string is non-empty
    if calday_in_str.strip("'"):  # checks if there's any date inside the quotes
        spark.sql(
            f"""
        DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_po_doctype
        WHERE calday IN ({calday_in_str})
        """
        ).display()
    else:
        print("no valid calday's found to delete.")
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dwh.f_po_doctype can't be deleted using calday's from temp_a_billcnt_store"
    )

In [0]:
if CALDAY_IN_STR != "":
    # Insert rows from temp table into main table
    INSERT_ROW = ", ".join(
        spark.table(f"{catalog_name}.udp_wcm_gold_vcm_dwh.f_po_doctype").columns
    )
    spark.sql(
        f"""
        INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.f_po_doctype ({INSERT_ROW})
        SELECT {INSERT_ROW} FROM temp_{target_table}
    """
    ).display()
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dwh.f_po_doctype can't be inserted using calday's from temp_{target_table}"
    )

In [0]:
spark.sql(f"""DROP VIEW IF EXISTS temp_{target_table}""")

In [0]:
if CALDAY_IN_STR == '':
    dbutils.notebook.exit("No valid calday's found In common-etl to insert data into Table: ETL_LOG_DEPENDENCIES and ETL_DELTA_TABLE and ")

In [0]:
%run "../common/common_etl_update"